# DEEP SPEECH1 구현 - pytorch

CTC loss 를 이용한 deep speech 구현이다. <br>
구현 모델은 [Deep speech 이론 by YBigTa](https://github.com/YBIGTA/Deep_learning/blob/master/RNN/deep%20speech/%EC%84%A4%EB%AA%85/Deep%20speech_%EC%83%81%ED%97%8C.pdf) 에서 확인 가능하다. <br>

본 코드는 [deep speech2 implementation](https://github.com/SeanNaren/deepspeech.pytorch/)을 상당부분 참고한다.

### 구현 stack
OS : ubuntu 16.04 <br>
conda : 4.2.9 <br>


## 설치

오디오 I/O 를 위한 pytorch audio를 설치한다. <br>
```
sudo apt-get install sox libsox-dev libsox-fmt-all
git clone https://github.com/pytorch/audio.git
cd audio
python setup.py install
```

Requirments를 설치한다. <br>
```
sudo pip install python-levenshtein torch visdom wget librosa
```

## 0. 데이터셋 다운로드

데이터셋의 경우 카네기 멜론 대학교에서 제공한 free dataset인 **AN4** 를 사용한다.

```python
import argparse
import os
import io
import shutil
import tarfile
import wget

import subprocess

def _order_files(file_paths):
    print("Sorting files by length...")

    def func(element):
        output = subprocess.check_output(
            ['soxi -D \"%s\"' % element.strip()],
            shell=True
        )
        return float(output)

    file_paths.sort(key=func)
    
def create_manifest(data_path, tag, ordered=True):
    manifest_path = '%s_manifest.csv' % tag
    file_paths = []
    wav_files = [os.path.join(dirpath, f)
                 for dirpath, dirnames, files in os.walk(data_path)
                 for f in fnmatch.filter(files, '*.wav')]
    size = len(wav_files)
    counter = 0
    for file_path in wav_files:
        file_paths.append(file_path.strip())
        counter += 1
    print('\n')
    if ordered:
        _order_files(file_paths)
    counter = 0
    with io.FileIO(manifest_path, "w") as file:
        for wav_path in file_paths:
            transcript_path = wav_path.replace('/wav/', '/txt/').replace('.wav', '.txt')
            sample = os.path.abspath(wav_path) + ',' + os.path.abspath(transcript_path) + '\n'
            file.write(sample.encode('utf-8'))
            counter += 1
    print('\n')

# command line에서 동작하는 것들을 더 쉽게 만들어주는 parser를 이용하여 데이터를 다운받는다.
parser = argparse.ArgumentParser(description='Processes and downloads an4.') 
parser.add_argument('--target_dir', default='an4_dataset/', help='Path to save dataset')
parser.add_argument('--sample_rate', default=16000, type=int, help='Sample rate')
args = parser.parse_args()


def _format_data(root_path, data_tag, name, wav_folder):
    data_path = args.target_dir + data_tag + '/' + name + '/'
    new_transcript_path = data_path + '/txt/'
    new_wav_path = data_path + '/wav/'

    os.makedirs(new_transcript_path)
    os.makedirs(new_wav_path)

    wav_path = root_path + 'wav/'
    file_ids = root_path + 'etc/an4_%s.fileids' % data_tag
    transcripts = root_path + 'etc/an4_%s.transcription' % data_tag
    train_path = wav_path + wav_folder

    _convert_audio_to_wav(train_path)
    _format_files(file_ids, new_transcript_path, new_wav_path, transcripts, wav_path)


def _convert_audio_to_wav(train_path):
    with os.popen('find %s -type f -name "*.raw"' % train_path) as pipe:
        for line in pipe:
            raw_path = line.strip()
            new_path = line.replace('.raw', '.wav').strip()
            cmd = 'sox -t raw -r %d -b 16 -e signed-integer -B -c 1 \"%s\" \"%s\"' % (
                args.sample_rate, raw_path, new_path)
            os.system(cmd)


def _format_files(file_ids, new_transcript_path, new_wav_path, transcripts, wav_path):
    with open(file_ids, 'r') as f:
        with open(transcripts, 'r') as t:
            paths = f.readlines()
            transcripts = t.readlines()
            for x in range(len(paths)):
                path = wav_path + paths[x].strip() + '.wav'
                filename = path.split('/')[-1]
                extracted_transcript = _process_transcript(transcripts, x)
                current_path = os.path.abspath(path)
                new_path = new_wav_path + filename
                text_path = new_transcript_path + filename.replace('.wav', '.txt')
                with io.FileIO(text_path, "w") as file:
                    file.write(extracted_transcript.encode('utf-8'))
                os.rename(current_path, new_path)


def _process_transcript(transcripts, x):
    extracted_transcript = transcripts[x].split('(')[0].strip("<s>").split('<')[0].strip().upper()
    return extracted_transcript


def main():
    root_path = 'an4/'
    name = 'an4'
    wget.download('http://www.speech.cs.cmu.edu/databases/an4/an4_raw.bigendian.tar.gz')
    tar = tarfile.open('an4_raw.bigendian.tar.gz')
    tar.extractall()
    os.makedirs(args.target_dir)
    _format_data(root_path, 'train', name, 'an4_clstk')
    _format_data(root_path, 'test', name, 'an4test_clstk')
    shutil.rmtree(root_path)
    os.remove('an4_raw.bigendian.tar.gz')
    train_path = args.target_dir + '/train/'
    test_path = args.target_dir + '/test/'
    print ('\n', 'Creating manifests...')
    create_manifest(train_path, 'an4_train')
    create_manifest(test_path, 'an4_val')


if __name__ == '__main__':
    main()
```

## 1.1 Making audio spectogram using fourier transformation

## Dataset label csv 파일 제작

### training, testing을 위한 data는 다음과 같은 형태로 폴더 내에 저장이 되어 있다. 

#### training 

*data/data_path_to_train/input* <br>
*data/data_path_to_train/label* <br>

#### testing

*data/data_path_to_test/input* <br>
*data/data_path_to_test/label* <br>

따라서 본 데이터를 load 하기 위하여 training 시, testing 시 각각 불러올 파일들의 path를 하나의 csv파일(manifest라 부른다.) 로 저장하면 부르기에 용이하다. <br>

위의 코드 중 create_manifest 함수가 본 파일을 csv화 해주는 함수이다. <br>

따라서 코드를 돌린 장소에 csv 파일을 참조한다.


## Dataset과 Dataloader 구현

pytorch의 dataset과 dataloader의 경우는 그 클래스가 정해져 있다. <br>

따라서 data loader와 dataset의 경우에는 사용자 정의 클래스를 만들기 위해서 **torch.utils.data**의 **Dataset**과 **DataLoader**를 상속받아야 한다. <br>

이를 통해 사용자 정의 dataset, dataloader를 구현할 것이며, <br>

본 dataset에는 audio spectogram 형태의 dataset이여야 하므로 *wav* 파일을 *spectogram* 형태로 바꾸는 함수를 집어넣고 구현한다., <br>

추가적인 dataset, dataloader 구현 튜토리얼은 [pytorch 공인 튜토리얼](http://pytorch.org/tutorials/beginner/data_loading_tutorial.html) 을 참고한다.

In [16]:
# 필요 class load
import librosa
import numpy as np
import scipy.signal
import torch
import torchaudio
from torch.utils.data import DataLoader
from torch.utils.data import Dataset


In [18]:
"""
wav 파일을 load하는 함수.
torchaudio를 사용한다. 

Input : 
    wav 파일 path. 자료형 : str
Output :
    오디오 파일의 numpy 형태. 자료형 : np
"""
def load_audio(path):
    sound, _ = torchaudio.load(path)
    sound = sound.numpy()
    if len(sound.shape) > 1:
        if sound.shape[1] == 1:
            sound = sound.squeeze()
        else:
            sound = sound.mean(axis=1)  # multiple channels, average
    return sound

In [19]:
"""
Fourier transformation parser.
spectogramparser는 spectogramDataset에 상속되며, 모든 wav 파일을 spectogram으로 파싱하는 클래스이다.

Param : 
    audio_conf - 오디오 특성의 딕셔너리. spectogram화를 위한 window 방식, window size, stride, 음성의 rate 가 들어있어야 한다. 자료형 : dict
"""
class SpectogramParser(object):
    # 초기화 함수. 
    def __init__(self, audio_conf):
        super(SpectogramParser, self).__init__()
        self.window_stride = audio_conf['window_stride']
        self.window_size = audio_conf['window_size']
        self.sample_rate = audio_conf['sample_rate']
        self.window = audio_conf['window']
    
    """
    parsing 함수. 
    Input : 
        audio_path. 자료형 : dict
    output : 
        해당 path의 spectogram, 자료형 : FloatTensor (MHz + 1, len)
    """
    def parse_audio(self, audio_path):
        
        y = load_audio(audio_path)
        n_fft = int(self.sample_rate * self.window_size)
        win_length = n_fft
        hop_length = int(self.sample_rate * self.window_stride)
        # STFT
        D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length,
                         win_length=win_length, window=self.window)
        spect, phase = librosa.magphase(D)
        # S = log(S+1)
        spect = np.log1p(spect)
        spect = torch.FloatTensor(spect)
        return spect

#### *해당 클래스 example*

In [59]:
# sample.wav 파일로 테스트. 본 파일은 an4 training set의 첫 번째 data이다.
audio_conf = {}
audio_conf["sample_rate"] = torchaudio.load("./sample.wav")[1] #torchaudio의 2번째 return value는 해당 wav 파일의 rate이다.
audio_conf["window_size"] = 0.02
audio_conf["window_stride"] = 0.01
audio_conf["window"] = scipy.signal.hamming

#class 초기화
parser =SpectogramParser(audio_conf)

#parsing된 데이터
parser.parse_audio("./sample.wav")


 18.7441  17.1461  17.6008  ...   18.2615  11.8843  17.3191
 17.9347  18.3567  18.4088  ...   18.1678  18.2587  15.9493
 17.3988  17.9348  18.1413  ...   18.9098  17.7493  17.6598
           ...               ⋱              ...            
 15.4377  17.1619  15.5005  ...   15.3234  15.6830  15.5769
 16.3273  16.5899  14.5669  ...   15.5692  15.3346  14.6444
 16.0336  16.2428  15.1358  ...   15.8261  14.1488  13.8953
[torch.FloatTensor of size 442x55]

## Data loader 구현

In [25]:
"""
pytorch의 dataset 구현. torch의 Dataset, SpectogramParser를 상속받는다.
또한 *parse_transcript* 함수를 보면, 해당 text 가 index로 리턴이 되어야 한다. 이 또한 Data loader에 구현이 되는 것이다.

Input :
    audio_conf - 부모 클래스 spectogramparser 초기화를 위한 인자. 자료형 : dict
    manifest_filepath - wav, input 으로 나뉘어져 있는 데이터의 경로가 저장되어있는 csv 파일. 자료형 : str
    labels - 우리가 관측하고 싶은 character들. 자료형 : str
"""
class SpectogramDataset(Dataset, SpectogramParser):
    #초기화 함수. label을 index로 변환하기 위한 딕셔너리 형태인 labels_map을 만들어야 한다.
    def __init__(self, audio_conf, manifest_filepath, labels):
        with open(manifest_filepath) as f:
            ids = f.readlines()
        ids = [x.strip().split(',') for x in ids]
        self.ids = ids
        self.size = len(ids)
        self.labels_map = dict([(labels[i], i) for i in range(len(labels))])
        super(SpectogramDataset, self).__init__(audio_conf)

    def __getitem__(self, index):
        sample = self.ids[index]
        audio_path, transcript_path = sample[0], sample[1]
        spect = self.parse_audio(audio_path)
        transcript = self.parse_transcript(transcript_path)
        return spect, transcript

    def parse_transcript(self, transcript_path):
        with open(transcript_path, 'r') as transcript_file:
            transcript = transcript_file.read().replace('\n', '')
        transcript = list(filter(None, [self.labels_map.get(x) for x in list(transcript)]))
        return transcript

    def __len__(self):
        return self.size

In [30]:
with open("an4_train_manifest.csv") as f:
    ids = f.readlines()

In [33]:
ids = [x.strip().split(',') for x in ids]

In [35]:
labels = "_'ABCDEFGHIJKLMNOPQRSTUVWXYZ "
labels_map = dict([(labels[i], i) for i in range(len(labels))])

In [37]:
parser =SpectogramParser(audio_conf)

In [38]:
audio_path = ids[0][0]
transcript_path = ids[0][1]

In [39]:
audio_path

'/home/mappiness/Desktop/deep learning/Deep_learning/RNN/deep speech/구현/an4_dataset/train/an4/wav/an253-fash-b.wav'

#### *해당 클래스 example*

In [26]:
# sample.wav 파일로 테스트. 본 파일은 an4 training set의 첫 번째 data이다.
audio_conf = {}
audio_conf["sample_rate"] = torchaudio.load("./sample.wav")[1] #torchaudio의 2번째 return value는 해당 wav 파일의 rate이다.
audio_conf["window_size"] = 0.02
audio_conf["window_stride"] = 0.01
audio_conf["window"] = scipy.signal.hamming

#초기화
dataset = SpectogramDataset(audio_conf,"an4_train_manifest.csv", "_'ABCDEFGHIJKLMNOPQRSTUVWXYZ ")

In [28]:
dataset[0]

UnicodeEncodeError: 'ascii' codec can't encode characters in position 68-69: ordinal not in range(128)

## 2. modeling


In [3]:
import math
from collections import OrderedDict
import torch
import torch.nn as nn
import torch.nn.functional as F

"""
RNN 모델상에서의 사용되는 4번째 layer RNN.
deep speech(2 아닌 1)논문에서는 bidirectional RNN을 사용한다.
"""
supported_rnns = {
    'lstm': nn.LSTM,
    'rnn': nn.RNN,
    'gru': nn.GRU
}
supported_rnns_inv = dict((v, k) for k, v in supported_rnns.items()) #내가봤을 때 inverse는 딱히 필요없어.

In [7]:
"""
Collapses input of dim T*N*H to (T*N)*H, and applies to a module.
Allows handling of variable sequence lengths and minibatch sizes.
:param module: Module to apply input to.
"""
"""
그니까 이게 minibatch를 하나의 string으로 만들어준다는 거지.
Q.  왜 굳이 그렇게하지? 
그냥 long sentence training을 위하여?
"""
class SequenceWise(nn.Module):
    def __init__(self, module):
        super(SequenceWise, self).__init__()
        self.module = module

    def forward(self, x):
        t, n = x.size(0), x.size(1)
        x = x.view(t * n, -1)
        x = self.module(x)
        x = x.view(t, n, -1)
        return x

    def __repr__(self):
        tmpstr = self.__class__.__name__ + ' (\n'
        tmpstr += self.module.__repr__()
        tmpstr += ')'
        return tmpstr
    
"""
이건 굉장히 typical 한 minibatch softmax.
각 data의 softmax를 torch.stack을 해야한다는거지.
Q. 근데 여기서 self.training은 어디서 오는거야
"""
class InferenceBatchSoftmax(nn.Module):
    def forward(self, input_):
        if not self.training:
            batch_size = input_.size()[0]
            return torch.stack([F.log_softmax(input_[i]) for i in range(batch_size)], 0)
        else:
            return input_
        
"""
batch구현
왜 batch normalization data를 sequencewise를 이 단에서 구현하느냐.
batch normalization에 대한 ref) https://shuuki4.wordpress.com/2016/01/13/batch-normalization-%EC%84%A4%EB%AA%85-%EB%B0%8F-%EA%B5%AC%ED%98%84/
이 sequencewise를 없애고 해보자 나중에.
"""
class BatchRNN(nn.Module):
    def __init__(self, input_size, hidden_size, rnn_type=nn.LSTM, bidirectional=False, batch_norm=True):
        super(BatchRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bidirectional = bidirectional
        self.batch_norm = SequenceWise(nn.BatchNorm1d(input_size)) if batch_norm else None
        self.rnn = rnn_type(input_size=input_size, hidden_size=hidden_size,
                            bidirectional=bidirectional, bias=False)
        self.num_directions = 2 if bidirectional else 1

    # Q. 그니까 RNN 의 hidden layer를 output으로 내는데, bidirectional의 경우에는 이 둘을 그냥 더한다?
    def forward(self, x):
        if self.batch_norm is not None:
            x = self.batch_norm(x)
        x, _ = self.rnn(x)
        if self.bidirectional:
            x = x.view(x.size(0), x.size(1), 2, -1).sum(2).view(x.size(0), x.size(1), -1)  # (TxNxH*2) -> (TxNxH) by sum
            return x

In [ ]:

"""
Q. labels의 역할은? 왜 여긴 abc밖에 없는거지?
Q. audio_conf는?
"""
class DeepSpeech(nn.Module):
    def __init__(self, rnn_type=nn.LSTM, labels="abc", rnn_hidden_size=768, nb_layers=5, audio_conf=None,
                 bidirectional=True):
        super(DeepSpeech, self).__init__()

        # model metadata needed for serialization/deserialization
        if audio_conf is None:
            audio_conf = {}
        self._version = '0.0.1'
        self._hidden_size = rnn_hidden_size
        self._hidden_layers = nb_layers
        self._rnn_type = rnn_type
        self._audio_conf = audio_conf or {}
        self._labels = labels

        sample_rate = self._audio_conf.get("sample_rate", 16000)
        window_size = self._audio_conf.get("window_size", 0.02)
        num_classes = len(self._labels)

        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(41, 11), stride=(2, 2)),
            nn.BatchNorm2d(32),
            nn.Hardtanh(0, 20, inplace=True),
            nn.Conv2d(32, 32, kernel_size=(21, 11), stride=(2, 1)),
            nn.BatchNorm2d(32),
            nn.Hardtanh(0, 20, inplace=True)
        )
        # Based on above convolutions and spectrogram size using conv formula (W - F + 2P)/ S+1
        rnn_input_size = int(math.floor((sample_rate * window_size) / 2) + 1)
        rnn_input_size = int(math.floor(rnn_input_size - 41) / 2 + 1)
        rnn_input_size = int(math.floor(rnn_input_size - 21) / 2 + 1)
        rnn_input_size *= 32

        rnns = []
        rnn = BatchRNN(input_size=rnn_input_size, hidden_size=rnn_hidden_size, rnn_type=rnn_type,
                       bidirectional=bidirectional, batch_norm=False)
        rnns.append(('0', rnn))
        for x in range(nb_layers - 1):
            rnn = BatchRNN(input_size=rnn_hidden_size, hidden_size=rnn_hidden_size, rnn_type=rnn_type,
                           bidirectional=bidirectional)
            rnns.append(('%d' % (x + 1), rnn))
        self.rnns = nn.Sequential(OrderedDict(rnns))
        fully_connected = nn.Sequential(
            nn.BatchNorm1d(rnn_hidden_size),
            nn.Linear(rnn_hidden_size, num_classes, bias=False)
        )
        self.fc = nn.Sequential(
            SequenceWise(fully_connected),
        )
        self.softmax = InferenceBatchSoftmax()

    def forward(self, x):
        x = self.conv(x)

        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # Collapse feature dimension
        x = x.transpose(1, 2).transpose(0, 1).contiguous()  # TxNxH

        x = self.rnns(x)

        x = self.fc(x)
        x = x.transpose(0, 1)
        x = self.softmax(x)
        return x

    @classmethod
    def load_model(cls, path, cuda=False):
        package = torch.load(path, map_location=lambda storage, loc: storage)
        model = cls(rnn_hidden_size=package['hidden_size'], nb_layers=package['hidden_layers'],
                    labels=package['labels'], audio_conf=package['audio_conf'],
                    rnn_type=supported_rnns[package['rnn_type']])
        model.load_state_dict(package['state_dict'])
        if cuda:
            model = torch.nn.DataParallel(model).cuda()
        return model

    @staticmethod
    def serialize(model, optimizer=None, epoch=None, iteration=None, loss_results=None,
                  cer_results=None, wer_results=None, avg_loss=None, meta=None):
        model_is_cuda = next(model.parameters()).is_cuda
        model = model.module if model_is_cuda else model
        package = {
            'version': model._version,
            'hidden_size': model._hidden_size,
            'hidden_layers': model._hidden_layers,
            'rnn_type': supported_rnns_inv.get(model._rnn_type, model._rnn_type.__name__.lower()),
            'audio_conf': model._audio_conf,
            'labels': model._labels,
            'state_dict': model.state_dict()
        }
        if optimizer is not None:
            package['optim_dict'] = optimizer.state_dict()
        if avg_loss is not None:
            package['avg_loss'] = avg_loss
        if epoch is not None:
            package['epoch'] = epoch + 1  # increment for readability
        if iteration is not None:
            package['iteration'] = iteration
        if loss_results is not None:
            package['loss_results'] = loss_results
            package['cer_results'] = cer_results
            package['wer_results'] = wer_results
        if meta is not None:
            package['meta'] = meta
        return package

    @staticmethod
    def get_labels(model):
        model_is_cuda = next(model.parameters()).is_cuda
        return model.module._labels if model_is_cuda else model._labels

    @staticmethod
    def get_param_size(model):
        params = 0
        for p in model.parameters():
            tmp = 1
            for x in p.size():
                tmp *= x
            params += tmp
        return params

    @staticmethod
    def get_audio_conf(model):
        model_is_cuda = next(model.parameters()).is_cuda
        return model.module._audio_conf if model_is_cuda else model._audio_conf



In [2]:
len("abc")

3